In [5]:
from requests import get 
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
from tqdm import tqdm_notebook

In [11]:
## Function for city
def getCity(soup):
    try:
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+\-\w+',q)[0]
        print(city)
    except: 
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+',q)[0]
    return city    

In [12]:
## Function for price

def getPrice(soup):
    obj=soup.find('div',{'style':'margin: 10px 0 0; padding-left: 21px; position: relative;'})
    obj=obj.text
    x=re.search('Видео-консультация',obj)
    if (x!=None):
        return [str(s) for s in obj.split() if s.isdigit()][0]
    else:
        return np.nan

In [13]:
## Function for demand - doesnt work
def getProperties(soup,stats):
    try:
        obj= soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'})
        qwe=obj.find('a',{'class':stats})
        qwe=qwe.text
        obj=[int(s) for s in qwe.split() if s.isdigit()]
    except: 
        obj=np.nan
    return obj

In [14]:
## Function for N publications
def publication(soup):
    try:
        pub=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_article'}).find('span',{'class':'n'})
        pub=pub.text
    except:
        pub=0
    return pub  

In [15]:
def feedback(soup):
    try:
        fd=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_otzyv'}).find('span',{'class':'n'})
        fd=fd.text
    except:
        fd=0
    return fd  

In [38]:
## Function for methods of work
def methods_work(soup):
    try:
        methods=soup.find('div',{'style':'margin: 20px 0;'}).text
        methods=re.split(r'\W+',methods)[3:]
        methods=' '.join(methods) #to convert from list to string
    except: methods=np.nan    
    return methods

In [17]:
## Function for marital status
def marital_status(soup):
    try:
        marital=soup.find_all('div',{'style':'margin: 10px 0;'})[2].text #several info with the same tags, parsed only MS (2)
        marital=re.split(r'\W+',marital)[2:]
        marital=' '.join(marital)
    except: marital=np.nan
    return marital

In [18]:
## Function for age
def age_con(soup):
    try:
        q=soup.find_all('div',{'style':'margin: 10px 0;'})[1].text
        q=re.findall('\d\d\d\d',q)
        q=''.join(q)
        q=2021-int(q)
    except: q=0    
    return q

In [19]:
## Function for education
def education(soup):
    edu=soup.find('div',{'id':'vuz'}).find('div',{'class':'vkladki_body_in vkladki_body_list'}).find('div',{'id':'vuz_list_s'}).text
    edu=list(edu.split("\n"))[:-1]
    return edu

In [39]:
def psych(sor):
    
    #Created html paper

    response = get(sor, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    #Apply function for demand
    demand=soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'}).find('a')
    demand=demand.text
    demand=re.findall("\d+",demand)[0]

    
    #Apply function for price
    price=getPrice(soup)
    
    #Extract city 
    city=getCity(soup)
    
    #Extract name
    name=soup.find('h1').text
    
    #Extract number of Publications 
    pub=publication(soup)
    
    #Extract number of Feedbacks
    otz=feedback(soup)
    
    #Extract description
    des= soup.find('div',{'class':'vkladki_body0'})
    des=des.find('p')
    des=des.text
    
    #Extract methods of work
    methods=methods_work(soup)
    
    #Extract marital status
    marital=marital_status(soup)
    
    #Extract age
    age=age_con(soup)
    
    #Extract education
    edu=education(soup)
    data_col={
        "name":name,
        "demand":demand,
        "price":price,
        "publications":pub,
        "otzivy":otz,
        "description":des,
        "city":city,
        "methods":methods,
        "marital_status":marital,
        "age":age,
        "education":edu
    }
    
    return data_col

In [40]:
data_col=psych('https://www.b17.ru/ivanivanovichkuzmin/')

г. Вольно-Надеждинское


In [41]:
df=pd.DataFrame(columns=["name","demand","price","publications","otzivy","description","city","methods","age","education"])

In [42]:
df=df.append(data_col,ignore_index=True)

In [43]:
df

,name,demand,price,publications,otzivy,description,city,methods,age,education,marital_status
0,Иван Кузьмин,683,NaN,161,57,Личный сайт: psy-ru-world.ruИнтернет-магазин (...,г. Вольно-Надеждинское,,44,[Саратовский военно-медицинский институт МО РФ...,Женат трое детей 1 внук Жена детский психолог ...


In [29]:
meme_page1 = 'https://www.b17.ru/online/?page=3'

response1 = get(meme_page1, headers={'User-Agent': UserAgent().chrome})

html1 = response1.content
soup1 = BeautifulSoup(html1,'html.parser')
response1 = get(meme_page1, headers={'User-Agent': UserAgent().chrome})

html1 = response1.content
soup1 = BeautifulSoup(html1,'html.parser')
x=soup1.find_all("a",{"class":"h"})[1:]
lina=['https://www.b17.ru'+link.get("href") for link in x]

In [23]:
lina

['https://www.b17.ru/tkolesnikova/',
 'https://www.b17.ru/fofanoff/',
 'https://www.b17.ru/bessarabova/',
 'https://www.b17.ru/ivanivanovichkuzmin/',
 'https://www.b17.ru/olgasvetlaya/',
 'https://www.b17.ru/id78011/',
 'https://www.b17.ru/helenburk/',
 'https://www.b17.ru/leo1967/',
 'https://www.b17.ru/ponachevnaya/',
 'https://www.b17.ru/eazhuravleva/']

In [30]:
for link in tqdm_notebook(lina):
    try:
        data_col=psych(link)
        df=df.append(data_col,ignore_index=True)
    except: 
        print(link)

<ipython-input-30-90fab032cdb0>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm_notebook(lina):


г. Вольно-Надеждинское
г. Франкфурт-на



In [31]:
df

,name,demand,price,publications,otzivy,description,city,methods,age,education,marital_status
0,Елена Журавлева,390,3000,7,23,"Меня зовут Елена, я - клинический психолог, а ...",г. Франкфурт-на,Арт терапия Когнитивная терапия Когнитивно пов...,53,"[Московский институт психоанализа г.Москва, Ря...",Замужем Состою в счастливом браке более 25 лет...
1,Колесникова Татьяна Сергеевна,593,4000,1737,197,Профессиональная сфера:- взаимоотношения в пар...,г. Нижний,Гештальт терапия Когнитивная терапия Когнитивн...,39,"[ННГУ им. Лобачевского, Организационная психол...",замужем
2,Фофанов Алексей Владимирович,494,1500,95,20,]Принципы и правила работы1.\tВ работе знакомл...,г. Биробиджан,Коучинг Логотерапия Нейро лингвистическое прог...,46,[Биробиджанский государственный педагогический...,NaN
3,Бессарабова Наталья Анатольевна,321,1500,409,10,В 2020 году прошла обучение в Международном ...,г. Черемхово,Арт терапия Коучинг Перинатальная психология П...,63,"[институт позитивных технологий и консалтинга,...",разведена
4,Иван Кузьмин,683,NaN,161,57,Личный сайт: psy-ru-world.ruИнтернет-магазин (...,г. Вольно-Надеждинское,,44,[Саратовский военно-медицинский институт МО РФ...,Женат трое детей 1 внук Жена детский психолог ...
5,Ольга Светлая,563,3000,340,45,До 3 апреля я в отпуске. Запись только на апре...,г. Краснодар,Гештальт терапия Гипнотерапия Коучинг Личностн...,38,[Московский институт аналитической психологии ...,замужем
6,Андрей Владимирович Бажажин,1144,1500,52,75,"Если Вы зашли на эту страницу, то, скорее всег...",г. Воронеж,Коучинг Личностно центрированная терапия Повед...,42,[Федеральное государственное бюджетное образов...,Женат
7,Буркова Елена Викторовна,453,3000,707,36,"Я являюсь клиническим психологом, магистром пс...",г. Челябинск,Когнитивно поведенческая терапия КПТ,39,[Южно-Уральский государственный гуманитарно-пе...,замужем дети есть
8,Леонид Ананьевич Казарин,169,1950,52,25,Детско-родительские отношения:- конфликты межд...,г. Беналмадена,Гештальт терапия Личностно центрированная тера...,54,[Нижегородский государственный университет име...,женат успешно воспитал троих воспитываю еще дв...
9,Татьяна Борисовна Поначевная,712,NaN,208,142,"Я психолог-практик, консультант, педагогСпециа...",г. Новороссийск,Когнитивно поведенческая терапия КПТ Символдра...,58,"[НОУДО "" Ангарская школа бизнеса, права и иску...",счастливая жена многодетная мама любимая бабушка


In [102]:
df.to_csv('data1.csv',index=False,encoding='UTF-8')